In [32]:
import tensorflow as tf

In [33]:
k, D, N = (3, 16, 320)

In [34]:
kernel_size = [k, k, D, N]
glorot_uni_initializer = tf.initializers.GlorotUniform()

kernels = tf.Variable(glorot_uni_initializer(kernel_size),
                      trainable=True, name="filters")
bias = tf.Variable(tf.zeros(shape=[N]), trainable=True, name='bias')

In [35]:
@tf.function
def conv_layer(x, kernels, bias, s):
    z = tf.nn.conv2d(x, kernels, strides=[1, s, s, 1], padding='VALID')
    return tf.nn.relu(z + bias)

In [36]:
class SimpleCNN(tf.keras.layers.Layer):
    def __init__(self, num_kernels=32, kernel_size=(3,3), stride=1):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernel_size = kernels_size
        self.stride = stride
        
    def build(self, input_shape):
        input_channels = input_shape[-1]
        
        kernels_shape = (*self.kernel_size, input_channels, self.num_kernels)
        glorot_init = tf.initializers.GlorotUniform()
        self.kernels = self.add_weight(name='kernels', shape = kernels_shape,
                                       initializer=glorot_init, trainable=True)
        self.bias = self.add_weight(name='bias', shape=(self.num_kernels, ),
                                    initializer='random_normal', trainable=True)
        
    def call(self, inputs):
        return conv_layer(inputs, self.kernels, self.bias, self.stride)

In [37]:
from tensorflow.keras.layers import Conv2D

In [38]:
s = 1

In [39]:
cov = Conv2D(filters=N, kernel_size=(k, k), strides=s,
             padding='valid', activation='relu')

In [40]:
from tensorflow.keras.layers import AvgPool2D, MaxPool2D

In [41]:
k, s = (3, 1)

In [42]:
avg_pool = AvgPool2D(pool_size=k, strides=[s, s], padding='valid')
max_pool = MaxPool2D(pool_size=k, strides=[s, s], padding='valid')

In [43]:
from tensorflow.keras.layers import Dense

In [44]:
output_size = 64

In [45]:
fc = Dense(units=output_size, activation='relu')

In [46]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.datasets import mnist

import numpy as np

In [47]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [48]:
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [49]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [50]:
print(x_train[0, :, :, 0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

In [51]:
x_train, x_test = x_train/255.0, x_test/255.0

In [52]:
print(x_train[0,:,:,0])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

In [53]:
num_classes = 10
epochs = 100
batch_size = 32

In [54]:
class LeNet5(Model):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu')
        self.conv2 = Conv2D(16, kernel_size=(5, 5), activation='relu')
        self.max_pool = MaxPool2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, input_data):
        x = self.max_pool(self.conv1(input_data))
        x = self.max_pool(self.conv2(x))
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x)))
        
        return x

In [55]:
model = LeNet5(num_classes)

In [57]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
             tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1)]

In [59]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          callbacks=callbacks)

Epoch 1/100
1875/1875 [==============================] - 11s 3ms/step - loss: 0.6339 - accuracy: 0.7983 - val_loss: 0.1737 - val_accuracy: 0.9465
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1284 - accuracy: 0.9610 - val_loss: 0.0775 - val_accuracy: 0.9762
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0869 - accuracy: 0.9736 - val_loss: 0.0597 - val_accuracy: 0.9793
Epoch 4/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0719 - accuracy: 0.9774 - val_loss: 0.0578 - val_accuracy: 0.9819
Epoch 5/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0612 - accuracy: 0.9806 - val_loss: 0.0550 - val_accuracy: 0.9819
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0533 - accuracy: 0.9837 - val_loss: 0.0385 - val_accuracy: 0.9864
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0474 - accuracy: 0.9849 - val_loss: 0.0381 - val_a

In [60]:
%load_ext tensorboard

In [62]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 14224), started 0:01:55 ago. (Use '!kill 14224' to kill it.)

In [63]:
vgg_net = tf.keras.applications.VGG16(include_top=True, weights='imagenet',
                                      input_tensor=None, input_shape=None,
                                      pooling=None, classes=1000)

553476096/553467096 [==============================] - 45s 0us/step


In [64]:
vgg_net.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [65]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input

In [66]:
input_shape = (28, 28, 3)

In [67]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5), input_shape=input_shape))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        2432      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                                 
 dense_5 (Dense)             (None, 10)                46090     
                                                                 
Total params: 48,522
Trainable params: 48,522
Non-trainable params: 0
_________________________________________________________________


In [69]:
def build_model():
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, kernel_size=(5,5))(inputs)
    maxpool1 = MaxPool2D(pool_size=(2,2))(conv1)
    predictions = Dense(10, activation='softmax')(Flatten()(maxpool1))
    
    model = Model(inputs=inputs, outputs=predictions)
    return model

In [70]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, concatenate

In [71]:
def naive_inception_block(prev_layer, filters=[64, 128, 32]):
    conv1x1 = Conv2D(filters[0], kernel_size=(1,1), padding='same', activation='relu')(prev_layer)
    conv3x3 = Conv2D(filters[1], kernel_size=(3,3), padding='same', activation='relu')(prev_layer)
    conv5x5 = Conv2D(filters[2], kernel_size=(5,5), padding='same', activation='relu')(prev_layer)
    max_pool = MaxPool2D((3,3), strides=(1,1), padding='same')(prev_layer)
    
    return concatenate([conv1x1, conv3x3, conv5x5, max_pool], axis=-1)

In [73]:
inceptionV3_net = tf.keras.applications.InceptionV3(
    include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
    pooling=None, classes=1000)

96124928/96112376 [==============================] - 8s 0us/step


In [74]:
inceptionV3_net.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d_6[0][0]']               
 alization)                     )                                                      

잔차 블록 구현(ResNet)

In [75]:
from tensorflow.keras.layers import Activation, Conv2D, BatchNormalization, add

In [ ]:
def residual_block_basic(x, filters, kernel_size=3, strides=1):
    conv_1 = Conv2D(filters=filters, kernel_size=kernel_size,
                    padding='same', strides=strides)(x)
    bn_1 = BatchNormalization(axis=-1)(conv_1)
    act_1 = Activation('relu')(bn_1)
    conv_2 = Conv2D(filter=filters, kernel_size=kernel_size,
                    padding='same', strides=strides)(act_1)
    residual = BatchNormalization(axis=-1)(conv_2)
    
    shortcut = x if strides == 1 else Conv2D(filters, kernel_size=1, padding='valid', strides=strides)(x)
    
    return Activation('relu')(add([shortcut, residual]))

케라스 모델

In [77]:
resnet50 = tf.keras.applications.ResNet50(
    include_top=True, weights='imagenet',
    input_tensor=None, input_shape=None,
    pooling=None, classes=1000)

102981632/102967424 [==============================] - 8s 0us/step


In [78]:
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           